## Scope

This notebook goes through how to create a training set from a set of questions and answers

## Initialize

In [2]:
import numpy as np

import time
import math
import sys
import pickle
import glob
import os
import tensorflow as tf


# Custom libraries
# from seq2seq_model import Seq2SeqModel
# from corpora_tools import *

## Import Data

In [8]:
# Option 1: All responses are equal to the original query except for the comment by the author
# Zipped Q/A
data = pickle.load(open('../data/all_responses_equal.p','rb'))
for i,j in list(data)[:4]:
    print(i,j)
    print()

{'author': 'RissaWasTaken', 'reader': 'kql', 'utterance': 'Husband deteriorating before my eyes, doctors at a loss, no one will help; Reddit docs, I need you.'} {'author': 'kql', 'reader': 'RissaWasTaken', 'utterance': "Hey, how's your husband doing now? Hope everything is okay."}

{'author': 'RissaWasTaken', 'reader': 'kql', 'utterance': 'Husband deteriorating before my eyes, doctors at a loss, no one will help; Reddit docs, I need you.'} {'author': 'kql', 'reader': 'RissaWasTaken', 'utterance': "I really do wish the best for your husband and hope you eventually get through this! keep your and your husband's head up and tell him he has people cheering for him sure, including me! :)"}

{'author': 'RissaWasTaken', 'reader': 'BrownIRL', 'utterance': 'Husband deteriorating before my eyes, doctors at a loss, no one will help; Reddit docs, I need you.'} {'author': 'BrownIRL', 'reader': 'RissaWasTaken', 'utterance': "A few weeks ago I was suffering from the same symptoms. Severe groin pain t

In [1]:
sen_l1, sen_l2 = retrieve_cornell_corpora()
print("# Two consecutive sentences in a conversation")
print("Q:", sen_l1[0])
print("A:", sen_l2[0])
print("# Corpora length (i.e. number of sentences)")
print(len(sen_l1))
assert len(sen_l1) == len(sen_l2)

NameError: name 'retrieve_cornell_corpora' is not defined

## Create Trainable dataset

In [1]:
def get_tokenized_sequencial_sentences(list_of_lines, line_text):
    for line in list_of_lines:
        for i in range(len(line) - 1):
            yield (line_text[line[i]].split(" "), line_text[line[i+1]].split(" "))

In [ ]:
def retrieve_cornell_corpora(storage_path="/tmp", storage_dir="cornell_movie_dialogs_corpus"):
   download_and_decompress("http://www.cs.cornell.edu/~cristian/data/cornell_movie_dialogs_corpus.zip",      
                     storage_path,
                           storage_dir)
   conversations = read_conversations(storage_path, storage_dir)
   lines = read_lines(storage_path, storage_dir)
   return tuple(zip(*list(get_tokenized_sequencial_sentences(conversations, lines))))

In [ ]:
clean_sen_l1 = [clean_sentence(s) for s in sen_l1]
clean_sen_l2 = [clean_sentence(s) for s in sen_l2]
filt_clean_sen_l1, filt_clean_sen_l2 = filter_sentence_length(clean_sen_l1, clean_sen_l2)
print("# Filtered Corpora length (i.e. number of sentences)")
print(len(filt_clean_sen_l1))
assert len(filt_clean_sen_l1) == len(filt_clean_sen_l2)